In [22]:
# paso 1, buscar la informacion del calendario
import time
import datetime
import pandas as pd
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# **Configuración del navegador**
options = Options()
options.add_argument("--disable-blink-features=AutomationControlled")  # Evitar detección como bot
options.add_argument("--log-level=3")  # Reducir logs en consola
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("disable-blink-features=AutomationControlled")

# ***Establecer un User-Agent personalizado***
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
options.add_argument(f"user-agent={user_agent}")

# **Inicializar WebDriver**
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# **Abrir la URL de FIBA**
url = 'https://www.fiba.basketball/es/events/womens-basketball-league-americas-2024/games'
driver.get(url)

# **Extraer el nombre del evento desde la URL**
event_name = url.split("/es/events/")[1].split("/games")[0]
print(f"🏆 Evento extraído: {event_name}")

# **Esperar a que el calendario cargue completamente**
time.sleep(10)

print("✅ Página abierta correctamente. Esperando que el contenido cargue...")

# **Buscar las fechas disponibles**
try:
    date_buttons = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, '//*[@data-testid="ui-calendar-day"]'))
    )
    print(f"✅ Número de fechas en el calendario: {len(date_buttons)}")
except Exception as e:
    print(f"❌ ERROR al extraer las fechas: {str(e)}")

# **Paso 2: Extraer información de cada fecha y partidos**
data = []
for index, button in enumerate(date_buttons):
    try:
        button.click()
        time.sleep(5)  # Esperar la carga de partidos

        # Extraer la fecha de la jornada con manejo de errores
        try:
            anio_xpath = f'//*[@id="themeWrapper"]/div[2]/div/div[1]/div[2]/div/div[2]/div/div/div[2]/div[2]/button[{index + 1}]/div[1]/div'
            mes_xpath = f'//*[@id="themeWrapper"]/div[2]/div/div[1]/div[2]/div/div[2]/div/div/div[2]/div[2]/button[{index + 1}]/div[4]'
            dia_xpath = f'//*[@id="themeWrapper"]/div[2]/div/div[1]/div[2]/div/div[2]/div/div/div[2]/div[2]/button[{index + 1}]/div[3]'

            anio = driver.find_element(By.XPATH, anio_xpath).text.strip()
            mes = driver.find_element(By.XPATH, mes_xpath).text.strip()
            dia = driver.find_element(By.XPATH, dia_xpath).text.strip()
            fecha = f"{anio}/{mes}/{dia}"
        except Exception as e:
            print(f"❌ ERROR al extraer fecha: {str(e)}")
            fecha = "Desconocida"

        print(f"📅 Procesando partidos del {fecha}")

        # **Extraer información de los partidos con manejo de errores**
        try:
            event_container = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.XPATH, '//*[@data-testid="event-games-listing"]'))
            )
            matches = event_container.find_elements(By.TAG_NAME, 'a')

            for match in matches:
                try:
                    fase = match.find_element(By.XPATH, './div/div[1]/div[1]').text.strip()
                    team_a = match.find_element(By.XPATH, './div/div[2]/div/div[1]/div[1]').text.strip()
                    team_b = match.find_element(By.XPATH, './div/div[2]/div/div[2]/div[1]').text.strip()
                    game_url = match.get_attribute("href")
                except Exception as e:
                    print(f"❌ ERROR al extraer datos del partido: {str(e)}")
                    fase, team_a, team_b, game_url = "Desconocida", "Desconocido", "Desconocido", "Sin URL"

                print(f"🏀 {fecha} - {fase}: {team_a} vs {team_b} - URL: {game_url}")

                data.append({
                    "Event": event_name,
                    "Fecha": fecha,
                    "Fase": fase,
                    "Team A": team_a,
                    "Team B": team_b,
                    "URL": game_url
                })
        except Exception as e:
            print(f"❌ ERROR al extraer lista de partidos: {str(e)}")

    except Exception as e:
        print(f"❌ ERROR al procesar fecha: {str(e)}")

# **Guardar df_calendar en un directorio separado 'calendar'**
calendar_dir = './calendarios'
os.makedirs(calendar_dir, exist_ok=True)

# **Guardar el archivo df_calendar en el directorio 'calendar'**
if data:
    df = pd.DataFrame(data)

    # **Generar la ruta del archivo con la fecha actual**
    fecha_actual = datetime.datetime.now().strftime("%Y-%m-%d")
    file_path = os.path.join(calendar_dir, f"df_calendar_{fecha_actual}.csv")

    try:
        df.to_csv(file_path, index=False, encoding='utf-8-sig')
        print(f"📁 Archivo guardado en: {file_path} calendario del evento")
    except Exception as e:
        print(f"❌ ERROR al guardar el archivo: {str(e)}")
else:
    print("⚠️ No se encontraron partidos para guardar.")


🏆 Evento extraído: womens-basketball-league-americas-2024
✅ Página abierta correctamente. Esperando que el contenido cargue...
✅ Número de fechas en el calendario: 8
❌ ERROR al extraer fecha: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="themeWrapper"]/div[2]/div/div[1]/div[2]/div/div[2]/div/div/div[2]/div[2]/button[1]/div[1]/div"}
  (Session info: chrome=135.0.7049.96); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00928073+60707]
	GetHandleVerifier [0x009280B4+60772]
	(No symbol) [0x00750683]
	(No symbol) [0x00798660]
	(No symbol) [0x007989FB]
	(No symbol) [0x007E1022]
	(No symbol) [0x007BD094]
	(No symbol) [0x007DE824]
	(No symbol) [0x007BCE46]
	(No symbol) [0x0078C5D3]
	(No symbol) [0x0078D424]
	GetHandleVerifier [0x00B6BB53+2435075]
	GetHandleVerifier [0x00B670F3+2416035]
	GetHandleVerifier [0x00B8349C+253

In [23]:
# paso 3 : Boxscore de cada partido, resultado, fecha, Id_match

# 📂 Directorio donde se guardan los archivos del calendario
calendar_dir = './calendarios'
os.makedirs(calendar_dir, exist_ok=True)

# 📂 Directorio para boxscore
output_dir = "boxscore"
os.makedirs(output_dir, exist_ok=True)

# 📌 Extraer la URL de los partidos desde el archivo generado en PASO 2
calendar_file = sorted([f for f in os.listdir(calendar_dir) if f.startswith("df_calendar")])[-1]
df_calendar = pd.read_csv(os.path.join(calendar_dir, calendar_file))
urlList_pbp = df_calendar["URL"].tolist()

# 📌 Si no hay partidos, detener el proceso
if len(urlList_pbp) == 0:
    print("⚠️ No hay partidos para procesar en el PASO 3.")
    exit()

# 📌 Inicializar el navegador para PASO 3
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 📌 Iterar sobre todas las URLs y procesar cada partido
for i, url in enumerate(urlList_pbp):
    print(f"Procesando partido {i + 1} de {len(urlList_pbp)}")
    match_id = url.split("/")[-1]  # Extraer ID_Match
    print(f"📌 ID del partido: {match_id}")
    
    try:
        # Intentar acceder a la página
        driver.get(url + "#boxscore")
        time.sleep(5)

        # 📄 PASO 3.1: Extraer HTML con BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 🔍 PASO 3.2: Obtener los nombres reales de los equipos
        try:
            team_A_element = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div/div/div[3]/div[3]/div[1]/div[2]/div[1]/div/div/div/div[1]/label/div/span/div")
            team_B_element = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div/div/div[3]/div[3]/div[1]/div[2]/div[1]/div/div/div/div[2]/label/div/button/span/div")
            
            team_A = team_A_element.text.strip() if team_A_element else "Equipo A"
            team_B = team_B_element.text.strip() if team_B_element else "Equipo B"
            
            print(f"✔️ Equipos encontrados: {team_A} 🆚 {team_B}")
        except Exception as e:
            print(f"⚠️ Error al obtener nombres de equipos: {str(e)}")
            continue

        # Obtener la fecha desde el df_calendar
        fecha_partido = df_calendar.loc[df_calendar["URL"] == url, "Fecha"].values[0]
        print(f"📅 Fecha del partido: {fecha_partido}")
        
        # 🏆 PASO 3.3: Obtener estadísticas de los equipos
        try:
            print(f"🔹 Procesando estadísticas de {team_A}...")
            tablas = soup.find_all("table", class_="tupwnca")
            if not tablas:
                print(f"❌ No se encontraron estadísticas en {url}.")
                continue
            df_team_A = pd.read_html(str(tablas[0]))[0]
            df_team_A['Team'] = team_A
            df_team_A['Opp'] = team_B
        except Exception as e:
            print(f"❌ ERROR en la extracción de estadísticas de {team_A}: {str(e)}")
            continue

        # 🔄 PASO 3.4: Extraer estadísticas del equipo B
        try:
            print(f"🔹 Procesando estadísticas de {team_B}...")
            team_B_element.click()
            time.sleep(5)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            tablas = soup.find_all("table", class_="tupwnca")
            if not tablas:
                print(f"❌ No se encontraron estadísticas de {team_B} en {url}.")
                continue
            df_team_B = pd.read_html(str(tablas[0]))[0]
            df_team_B['Team'] = team_B
            df_team_B['Opp'] = team_A
        except Exception as e:
            print(f"❌ ERROR en la extracción de estadísticas de {team_B}: {str(e)}")
            continue

        # 🔢 Obtener el resultado del partido
        try:
            resultados = soup.find_all('div', class_='_1d88n031 japnsh4 _1d88n032 _1d88n033')
            textos = [resultado.get_text(strip=True) for resultado in resultados]
            resultado_A, resultado_B = textos[0].split(" - ") if " - " in textos[0] else textos[0].split("-")
            resultado_A, resultado_B = int(resultado_A), int(resultado_B)
            df_team_A['Resultado'] = "Victoria" if resultado_A > resultado_B else "Derrota"
            df_team_B['Resultado'] = "Victoria" if resultado_B > resultado_A else "Derrota"
        except Exception as e:
            print(f"⚠️ Error al obtener el resultado del partido: {str(e)}")

        # Unir los datos  de Team A y Team B
        df_final = pd.concat([df_team_A, df_team_B], ignore_index=True)
        df_final['Game_URL'] = url
        df_final['Fecha'] = fecha_partido
        df_final['Match_ID'] = match_id

        file_name = f"{output_dir}/{match_id}.csv"
        df_final.to_csv(file_name, index=False)
        print(f"✔️ Datos exportados para el partido {match_id}.")

    except Exception as e:
        print(f"⚠️ Error procesando el partido {url}: {str(e)}")

# 🚪 Cerrar navegador después del proceso
driver.quit()
print("✅ PASO 3 COMPLETADO. Boxscore, resultado, fecha, Id_match")


Procesando partido 1 de 16
📌 ID del partido: 118315-CUF-MAL
✔️ Equipos encontrados: Union Florida 🆚 Malvin
📅 Fecha del partido: Desconocida
🔹 Procesando estadísticas de Union Florida...
🔹 Procesando estadísticas de Malvin...


C:\Users\anavarro\AppData\Local\Temp\ipykernel_13356\1002073296.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_team_A = pd.read_html(str(tablas[0]))[0]
C:\Users\anavarro\AppData\Local\Temp\ipykernel_13356\1002073296.py:79: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_team_B = pd.read_html(str(tablas[0]))[0]


✔️ Datos exportados para el partido 118315-CUF-MAL.
Procesando partido 2 de 16
📌 ID del partido: 118314-FCO-SSI
✔️ Equipos encontrados: Ferro 🆚 Sportiva Italiana
📅 Fecha del partido: Desconocida
🔹 Procesando estadísticas de Ferro...
🔹 Procesando estadísticas de Sportiva Italiana...


C:\Users\anavarro\AppData\Local\Temp\ipykernel_13356\1002073296.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_team_A = pd.read_html(str(tablas[0]))[0]
C:\Users\anavarro\AppData\Local\Temp\ipykernel_13356\1002073296.py:79: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_team_B = pd.read_html(str(tablas[0]))[0]


✔️ Datos exportados para el partido 118314-FCO-SSI.
Procesando partido 3 de 16
📌 ID del partido: 118317-CUF-SSI
✔️ Equipos encontrados: Union Florida 🆚 Sportiva Italiana
📅 Fecha del partido: Desconocida
🔹 Procesando estadísticas de Union Florida...
🔹 Procesando estadísticas de Sportiva Italiana...


...

✔️ Datos exportados para el partido 119974-IND-CUF.
Procesando partido 15 de 16
📌 ID del partido: 119976-CUF-SSI
✔️ Equipos encontrados: Union Florida 🆚 Sportiva Italiana
📅 Fecha del partido: Desconocida
🔹 Procesando estadísticas de Union Florida...
🔹 Procesando estadísticas de Sportiva Italiana...


C:\Users\anavarro\AppData\Local\Temp\ipykernel_13356\1002073296.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_team_A = pd.read_html(str(tablas[0]))[0]
C:\Users\anavarro\AppData\Local\Temp\ipykernel_13356\1002073296.py:79: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_team_B = pd.read_html(str(tablas[0]))[0]


✔️ Datos exportados para el partido 119976-CUF-SSI.
Procesando partido 16 de 16
📌 ID del partido: 119977-IND-BAP
✔️ Equipos encontrados: Indeportes Antioquia 🆚 Phoenix
📅 Fecha del partido: Desconocida
🔹 Procesando estadísticas de Indeportes Antioquia...
🔹 Procesando estadísticas de Phoenix...


C:\Users\anavarro\AppData\Local\Temp\ipykernel_13356\1002073296.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_team_A = pd.read_html(str(tablas[0]))[0]
C:\Users\anavarro\AppData\Local\Temp\ipykernel_13356\1002073296.py:79: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_team_B = pd.read_html(str(tablas[0]))[0]


✔️ Datos exportados para el partido 119977-IND-BAP.
✅ PASO 3 COMPLETADO. Boxscore, resultado, fecha, Id_match


**Todos los boxscore del partido estan guardados en csv en la carpeta /boxscore** 

vamos a a trabajar con esa informacion

In [ ]:
# 🛠️ PASO 4: Cargar y Unificar los Archivos CSV de Boxscore


# 📂 Directorio donde están los archivos CSV individuales de cada partido
directorio = "boxscore"  # Ruta relativa a la carpeta del proyecto

# 🔹 Lista para almacenar los DataFrames de cada archivo CSV
dataframes = []
csv_count = 0  # Contador de archivos procesados

# 🔍 Iterar sobre todos los archivos CSV en el directorio
for filename in os.listdir(directorio):
    if filename.endswith('.csv'):
        filepath = os.path.join(directorio, filename)
        df = pd.read_csv(filepath, low_memory=False)  # Cargar archivo CSV
        dataframes.append(df)
        csv_count += 1

# 🔗 Concatenar todos los archivos en un solo DataFrame
boxscore_unificado = pd.concat(dataframes, ignore_index=True)

# ✅ Mostrar resumen de archivos cargados y primeras filas
print(f"✅ Se han concatenado {csv_count} archivos CSV desde la carpeta '{directorio}'.")
print(f"🔢 Total de registros unificados: {boxscore_unificado.shape[0]}")
display(boxscore_unificado.head(10))

# 🔹 Mostrar información sobre los archivos cargados
print(f"✅ Se han concatenado {csv_count} archivos CSV.")
print(boxscore_unificado.head(10))


✅ Se han concatenado 16 archivos CSV desde la carpeta 'boxscore'.
🔢 Total de registros unificados: 434


,#,Jugadores,MIN,PTS,TC,2PT TC,3PT TC,TL,OREB,DREB,...,ROB,TAP,+/-,EFC,Team,Opp,Resultado,Game_URL,Fecha,Match_ID
0,0,Melody Osuna,No jugó,No jugó,No jugó,No jugó,No jugó,No jugó,No jugó,No jugó,...,NaN,NaN,NaN,NaN,Ferro,Sportiva Italiana,Derrota,https://www.fiba.basketball/es/events/womens-b...,Desconocida,118314-FCO-SSI
1,1,Florencia Fernandez *,28:34,19,6/12(50%),6/12(50%),0/0(0%),7/9(77.8%),5,7,...,1.0,1.0,-13.0,24.0,Ferro,Sportiva Italiana,Derrota,https://www.fiba.basketball/es/events/womens-b...,Desconocida,118314-FCO-SSI
2,4,Stephanie Smidt,03:00,0,0/1(0%),0/0(0%),0/1(0%),0/0(0%),0,0,...,0.0,0.0,-7.0,-2.0,Ferro,Sportiva Italiana,Derrota,https://www.fiba.basketball/es/events/womens-b...,Desconocida,118314-FCO-SSI
3,5,Julia Fernandez *,16:32,4,2/6(33.3%),2/6(33.3%),0/0(0%),0/0(0%),1,3,...,2.0,0.0,-12.0,1.0,Ferro,Sportiva Italiana,Derrota,https://www.fiba.basketball/es/events/womens-b...,Desconocida,118314-FCO-SSI
4,7,Candela Foresto *,26:32,8,3/5(60%),1/3(33.3%),2/2(100%),0/0(0%),2,1,...,1.0,0.0,-6.0,12.0,Ferro,Sportiva Italiana,Derrota,https://www.fiba.basketball/es/events/womens-b...,Desconocida,118314-FCO-SSI
5,8,Mora Sauro,01:26,0,0/1(0%),0/1(0%),0/0(0%),0/0(0%),0,0,...,0.0,0.0,-1.0,-1.0,Ferro,Sportiva Italiana,Derrota,https://www.fiba.basketball/es/events/womens-b...,Desconocida,118314-FCO-SSI
6,11,Abril Romagnoli *,14:33,2,1/6(16.7%),1/2(50%),0/4(0%),0/0(0%),1,0,...,1.0,0.0,-12.0,-2.0,Ferro,Sportiva Italiana,Derrota,https://www.fiba.basketball/es/events/womens-b...,Desconocida,118314-FCO-SSI
7,13,Lucila Sampietro,19:09,0,0/11(0%),0/5(0%),0/6(0%),0/0(0%),1,1,...,1.0,0.0,-5.0,-9.0,Ferro,Sportiva Italiana,Derrota,https://www.fiba.basketball/es/events/womens-b...,Desconocida,118314-FCO-SSI
8,14,Maribel Barzola,20:15,3,1/5(20%),0/1(0%),1/4(25%),0/0(0%),0,4,...,1.0,0.0,-7.0,3.0,Ferro,Sportiva Italiana,Derrota,https://www.fiba.basketball/es/events/womens-b...,Desconocida,118314-FCO-SSI
9,16,Julieta Perez,14:39,7,2/5(40%),2/5(40%),0/0(0%),3/6(50%),3,7,...,1.0,0.0,-4.0,10.0,Ferro,Sportiva Italiana,Derrota,https://www.fiba.basketball/es/events/womens-b...,Desconocida,118314-FCO-SSI


✅ Se han concatenado 16 archivos CSV.
    #              Jugadores      MIN      PTS          TC      2PT TC  \
0   0           Melody Osuna  No jugó  No jugó     No jugó     No jugó   
1   1  Florencia Fernandez *    28:34       19   6/12(50%)   6/12(50%)   
2   4        Stephanie Smidt    03:00        0     0/1(0%)     0/0(0%)   
3   5      Julia Fernandez *    16:32        4  2/6(33.3%)  2/6(33.3%)   
4   7      Candela Foresto *    26:32        8    3/5(60%)  1/3(33.3%)   
5   8             Mora Sauro    01:26        0     0/1(0%)     0/1(0%)   
6  11      Abril Romagnoli *    14:33        2  1/6(16.7%)    1/2(50%)   
7  13       Lucila Sampietro    19:09        0    0/11(0%)     0/5(0%)   
8  14        Maribel Barzola    20:15        3    1/5(20%)     0/1(0%)   
9  16          Julieta Perez    14:39        7    2/5(40%)    2/5(40%)   

      3PT TC          TL     OREB     DREB  ...  ROB  TAP   +/-   EFC   Team  \
0    No jugó     No jugó  No jugó  No jugó  ...  NaN  NaN   NaN   N

In [25]:
# Paso 5: Verificar la Integridad de los Datos

# 🔹 Revisar columnas y tipos de datos
print("\n📌 Columnas en el dataset:")
print(boxscore_unificado.dtypes)

# 🔹 Revisar valores nulos
print("\n⚠️ Valores nulos en el dataset:")
print(boxscore_unificado.isnull().sum())

# 🔹 Verificar duplicados
print(f"\n⚠️ Filas duplicadas: {boxscore_unificado.duplicated().sum()}")

# 🔹 Ver los primeros 10 registros
print("\n📊 Primeras filas del dataset:")
print(boxscore_unificado.head(10))#



📌 Columnas en el dataset:
#             object
Jugadores     object
MIN           object
PTS           object
TC            object
2PT TC        object
3PT TC        object
TL            object
OREB          object
DREB          object
REB          float64
AST          float64
FP           float64
PER          float64
ROB          float64
TAP          float64
+/-          float64
EFC          float64
Team          object
Opp           object
Resultado     object
Game_URL      object
Fecha         object
Match_ID      object
dtype: object

⚠️ Valores nulos en el dataset:
#             0
Jugadores     0
MIN          32
PTS          32
TC           32
2PT TC       32
3PT TC       32
TL           32
OREB          0
DREB          0
REB          32
AST          64
FP           32
PER          32
ROB          64
TAP          64
+/-          96
EFC          32
Team          0
Opp           0
Resultado     0
Game_URL      0
Fecha         0
Match_ID      0
dtype: int64

⚠️ Filas duplicadas: 0



prueba de pasos 6 al 11


In [26]:


## prueba resultado
# 🛠️ **PASO 6: Limpieza de datos**
def limpiar_datos(df):
    """ Elimina filas irrelevantes, ajusta valores en MIN y limpia caracteres innecesarios sin modificar 'Resultado' ni 'Match_ID'. """
    
    df = df.copy()
    
    # Reemplazar valores de 'Jugadores'
    df.loc[df['Jugadores'] == 'Team/Coaches', 'Jugadores'] = 'Sin asignar'
    df = df[df['Jugadores'] != 'TOTAL']
    
    # Ajustar valores en 'MIN'
    df.loc[df['MIN'] == 'Did Not Play', 'MIN'] = '0'
    
    # Limpiar caracteres innecesarios
    df = df.map(lambda x: x.split('(')[0].strip() if isinstance(x, str) and '(' in x else x)

    # 📌 Asegurar que 'Resultado' NO SE MODIFIQUE
    df['Resultado'] = df['Resultado'].astype(str)  # Se mantiene en formato texto

    # 📌 Asegurar que 'Match_ID' se mantenga como texto
    df['Match_ID'] = df['Match_ID'].astype(str)

    print("\n✅ PASO 6 - Limpieza de datos completada.")
    print("📌 Columnas después de la limpieza:", list(df.columns))
    
    return df


# 🛠️ **PASO 7: Separación de columnas con '/'**
def separar_columnas(df):
    """ 
    - Identifica columnas con valores en formato 'X/Y'.
    - Divide estas columnas en '_MADE' y '_ATTEMPT'.
    - Se excluyen 'Game_URL', 'Fecha' y 'Match_ID' para evitar errores.
    """

    columns_to_split = [col for col in df.columns if col not in ['MIN', 'Game_URL', 'Fecha', 'Match_ID'] and df[col].astype(str).str.contains('/').any()]
    
    print("\n✅ PASO 7 - Separación de columnas iniciada.")
    print("🔹 Columnas a dividir:", columns_to_split)
    
    for col in columns_to_split:
        df[col] = df[col].astype(str).fillna("0/0")  # Reemplazar valores nulos con '0/0'
        df[[f"{col}_MADE", f"{col}_ATTEMPT"]] = df[col].str.split('/', expand=True)  # Crear nuevas columnas
    
    df.drop(columns=columns_to_split, inplace=True, errors='ignore')  # Eliminar columnas originales
    
    print("✅ PASO 7 - Separación de columnas completada.")
    return df

# 🛠️ **PASO 8: Conversión de columnas numéricas**
def convertir_columnas(df):
    """ 
    - Convierte columnas numéricas de tipo string a valores numéricos.
    """
    columnas_excluir = ['Jugadores', 'MIN', 'Team', 'Opp', 'Game_URL', "Resultado", 'Fecha', 'Match_ID']
    numeric_cols = df.select_dtypes(include=['object']).columns.drop(columnas_excluir, errors='ignore')
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
    df.fillna(0, inplace=True)
    
    print("\n✅ PASO 8 - Conversión de columnas numéricas completada.")
    print("📊 Columnas convertidas:", list(numeric_cols))
    return df

# 🛠️ **PASO 9: Conversión del formato de tiempo**
def convertir_formato_tiempo(df):
    """ Convierte la columna 'MIN' de formato 'MM:SS' a decimal """
    def convertir_minutos(time_str):
        if isinstance(time_str, str) and ':' in time_str:
            minutos, segundos = map(int, time_str.split(':'))
            return round(minutos + segundos / 60, 2)
        return float(time_str) if str(time_str).isdigit() else 0.0
    
    df['MIN'] = df['MIN'].apply(convertir_minutos)
    print("\n✅ PASO 9 - Conversión de tiempo (MIN) completada.")
    return df

# 🛠️ **PASO 10: Corrección del formato de fecha**
def convertir_formato_fecha(df):
    """ Convierte la columna 'Fecha' al formato DD/MM/AA """
    def convertir_fecha(fecha_str):
        partes = fecha_str.split('/')
        if len(partes) == 3:
            año, mes, dia = partes
            return f"{dia}/{mes}/{año[2:]}"
        return fecha_str
    
    if 'Fecha' in df.columns:
        df['Fecha'] = df['Fecha'].apply(convertir_fecha)
    
    print("\n✅ PASO 10 - Conversión de formato de fecha completada.")
    return df

# =============================
# 🛠️ PASO 6: Limpieza de Datos
# =============================
boxscore_limpio = limpiar_datos(boxscore_unificado)
print("\n✅ PASO 6 completado: Limpieza de datos")
print("📐 Dimensiones:", boxscore_limpio.shape)
display(boxscore_limpio.head(5))
# =============================
# 🛠️ PASO 7: Separación columnas X/Y
# =============================
boxscore_limpio = separar_columnas(boxscore_limpio)

print("\n✅ PASO 7 completado: Separación de columnas X/Y")
print("📐 Dimensiones:", boxscore_limpio.shape)
display(boxscore_limpio.head(5))
# =============================
# 🛠️ PASO 8: Conversión a columnas numéricas
# =============================
boxscore_limpio = convertir_columnas(boxscore_limpio)

print("\n✅ PASO 8 completado: Conversión de columnas numéricas")
print("📐 Dimensiones:", boxscore_limpio.shape)
display(boxscore_limpio.head(5))
# =============================
# 🛠️ PASO 9: Conversión del tiempo en 'MIN'
# =============================
boxscore_limpio = convertir_formato_tiempo(boxscore_limpio)

print("\n✅ PASO 9 completado: Conversión de tiempo en MIN a decimal")
print("📐 Dimensiones:", boxscore_limpio.shape)
display(boxscore_limpio.head(5))
# =============================
# 🛠️ PASO 10: Corrección de formato de fecha
# =============================
boxscore_limpio = convertir_formato_fecha(boxscore_limpio)

print("\n✅ PASO 10 completado: Formato de fecha ajustado (DD/MM/AA)")
print("📐 Dimensiones:", boxscore_limpio.shape)
display(boxscore_limpio.head(5))




✅ PASO 6 - Limpieza de datos completada.
📌 Columnas después de la limpieza: ['#', 'Jugadores', 'MIN', 'PTS', 'TC', '2PT TC', '3PT TC', 'TL', 'OREB', 'DREB', 'REB', 'AST', 'FP', 'PER', 'ROB', 'TAP', '+/-', 'EFC', 'Team', 'Opp', 'Resultado', 'Game_URL', 'Fecha', 'Match_ID']

✅ PASO 6 completado: Limpieza de datos
📐 Dimensiones: (402, 24)


,#,Jugadores,MIN,PTS,TC,2PT TC,3PT TC,TL,OREB,DREB,...,ROB,TAP,+/-,EFC,Team,Opp,Resultado,Game_URL,Fecha,Match_ID
0,0,Melody Osuna,No jugó,No jugó,No jugó,No jugó,No jugó,No jugó,No jugó,No jugó,...,NaN,NaN,NaN,NaN,Ferro,Sportiva Italiana,Derrota,https://www.fiba.basketball/es/events/womens-b...,Desconocida,118314-FCO-SSI
1,1,Florencia Fernandez *,28:34,19,6/12,6/12,0/0,7/9,5,7,...,1.0,1.0,-13.0,24.0,Ferro,Sportiva Italiana,Derrota,https://www.fiba.basketball/es/events/womens-b...,Desconocida,118314-FCO-SSI
2,4,Stephanie Smidt,03:00,0,0/1,0/0,0/1,0/0,0,0,...,0.0,0.0,-7.0,-2.0,Ferro,Sportiva Italiana,Derrota,https://www.fiba.basketball/es/events/womens-b...,Desconocida,118314-FCO-SSI
3,5,Julia Fernandez *,16:32,4,2/6,2/6,0/0,0/0,1,3,...,2.0,0.0,-12.0,1.0,Ferro,Sportiva Italiana,Derrota,https://www.fiba.basketball/es/events/womens-b...,Desconocida,118314-FCO-SSI
4,7,Candela Foresto *,26:32,8,3/5,1/3,2/2,0/0,2,1,...,1.0,0.0,-6.0,12.0,Ferro,Sportiva Italiana,Derrota,https://www.fiba.basketball/es/events/womens-b...,Desconocida,118314-FCO-SSI



✅ PASO 7 - Separación de columnas iniciada.
🔹 Columnas a dividir: ['#', 'TC', '2PT TC', '3PT TC', 'TL']
✅ PASO 7 - Separación de columnas completada.

✅ PASO 7 completado: Separación de columnas X/Y
📐 Dimensiones: (402, 29)


,Jugadores,MIN,PTS,OREB,DREB,REB,AST,FP,PER,ROB,...,#_MADE,#_ATTEMPT,TC_MADE,TC_ATTEMPT,2PT TC_MADE,2PT TC_ATTEMPT,3PT TC_MADE,3PT TC_ATTEMPT,TL_MADE,TL_ATTEMPT
0,Melody Osuna,No jugó,No jugó,No jugó,No jugó,NaN,NaN,NaN,NaN,NaN,...,0,None,No jugó,None,No jugó,None,No jugó,None,No jugó,None
1,Florencia Fernandez *,28:34,19,5,7,12.0,1.0,3.0,2.0,1.0,...,1,None,6,12,6,12,0,0,7,9
2,Stephanie Smidt,03:00,0,0,0,0.0,0.0,2.0,1.0,0.0,...,4,None,0,1,0,0,0,1,0,0
3,Julia Fernandez *,16:32,4,1,3,4.0,0.0,4.0,5.0,2.0,...,5,None,2,6,2,6,0,0,0,0
4,Candela Foresto *,26:32,8,2,1,3.0,3.0,2.0,1.0,1.0,...,7,None,3,5,1,3,2,2,0,0



✅ PASO 8 - Conversión de columnas numéricas completada.
📊 Columnas convertidas: ['PTS', 'OREB', 'DREB', '#_MADE', '#_ATTEMPT', 'TC_MADE', 'TC_ATTEMPT', '2PT TC_MADE', '2PT TC_ATTEMPT', '3PT TC_MADE', '3PT TC_ATTEMPT', 'TL_MADE', 'TL_ATTEMPT']

✅ PASO 8 completado: Conversión de columnas numéricas
📐 Dimensiones: (402, 29)


,Jugadores,MIN,PTS,OREB,DREB,REB,AST,FP,PER,ROB,...,#_MADE,#_ATTEMPT,TC_MADE,TC_ATTEMPT,2PT TC_MADE,2PT TC_ATTEMPT,3PT TC_MADE,3PT TC_ATTEMPT,TL_MADE,TL_ATTEMPT
0,Melody Osuna,No jugó,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Florencia Fernandez *,28:34,19.0,5.0,7.0,12.0,1.0,3.0,2.0,1.0,...,1.0,0.0,6.0,12.0,6.0,12.0,0.0,0.0,7.0,9.0
2,Stephanie Smidt,03:00,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,...,4.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Julia Fernandez *,16:32,4.0,1.0,3.0,4.0,0.0,4.0,5.0,2.0,...,5.0,0.0,2.0,6.0,2.0,6.0,0.0,0.0,0.0,0.0
4,Candela Foresto *,26:32,8.0,2.0,1.0,3.0,3.0,2.0,1.0,1.0,...,7.0,0.0,3.0,5.0,1.0,3.0,2.0,2.0,0.0,0.0



✅ PASO 9 - Conversión de tiempo (MIN) completada.

✅ PASO 9 completado: Conversión de tiempo en MIN a decimal
📐 Dimensiones: (402, 29)


,Jugadores,MIN,PTS,OREB,DREB,REB,AST,FP,PER,ROB,...,#_MADE,#_ATTEMPT,TC_MADE,TC_ATTEMPT,2PT TC_MADE,2PT TC_ATTEMPT,3PT TC_MADE,3PT TC_ATTEMPT,TL_MADE,TL_ATTEMPT
0,Melody Osuna,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Florencia Fernandez *,28.57,19.0,5.0,7.0,12.0,1.0,3.0,2.0,1.0,...,1.0,0.0,6.0,12.0,6.0,12.0,0.0,0.0,7.0,9.0
2,Stephanie Smidt,3.00,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,...,4.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Julia Fernandez *,16.53,4.0,1.0,3.0,4.0,0.0,4.0,5.0,2.0,...,5.0,0.0,2.0,6.0,2.0,6.0,0.0,0.0,0.0,0.0
4,Candela Foresto *,26.53,8.0,2.0,1.0,3.0,3.0,2.0,1.0,1.0,...,7.0,0.0,3.0,5.0,1.0,3.0,2.0,2.0,0.0,0.0



✅ PASO 10 - Conversión de formato de fecha completada.

✅ PASO 10 completado: Formato de fecha ajustado (DD/MM/AA)
📐 Dimensiones: (402, 29)


,Jugadores,MIN,PTS,OREB,DREB,REB,AST,FP,PER,ROB,...,#_MADE,#_ATTEMPT,TC_MADE,TC_ATTEMPT,2PT TC_MADE,2PT TC_ATTEMPT,3PT TC_MADE,3PT TC_ATTEMPT,TL_MADE,TL_ATTEMPT
0,Melody Osuna,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Florencia Fernandez *,28.57,19.0,5.0,7.0,12.0,1.0,3.0,2.0,1.0,...,1.0,0.0,6.0,12.0,6.0,12.0,0.0,0.0,7.0,9.0
2,Stephanie Smidt,3.00,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,...,4.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Julia Fernandez *,16.53,4.0,1.0,3.0,4.0,0.0,4.0,5.0,2.0,...,5.0,0.0,2.0,6.0,2.0,6.0,0.0,0.0,0.0,0.0
4,Candela Foresto *,26.53,8.0,2.0,1.0,3.0,3.0,2.0,1.0,1.0,...,7.0,0.0,3.0,5.0,1.0,3.0,2.0,2.0,0.0,0.0


In [27]:
# 🛠️ PASO 11: Renombrar columnas a nombres estándar
def renombrar_columnas(df):
    """ Cambiar nombres de columnas al formato estándar """
    diccionario_columnas = {
        'TC_MADE': 'TCC', 'TC_ATTEMPT': 'TCI',
        '2PT TC_MADE': '2PC', '2PT TC_ATTEMPT': '2PI',
        '3PT TC_MADE': '3PC', '3PT TC_ATTEMPT': '3PI',
        'TL_MADE': 'TLC', 'TL_ATTEMPT': 'TLI'
    }
    df.rename(columns=diccionario_columnas, inplace=True, errors="ignore")
    print("\n✅ PASO 11 - Renombrado de columnas completado.")
    return df

# 🚀 Aplicar renombramiento a boxscore_limpio
boxscore_limpio = renombrar_columnas(boxscore_limpio)

# 🧩 Crear alias requeridos por stats_advanced()
boxscore_limpio['FGM'] = boxscore_limpio['TCC']
boxscore_limpio['FGA'] = boxscore_limpio['TCI']
boxscore_limpio['FTM'] = boxscore_limpio['TLC']
boxscore_limpio['FTA'] = boxscore_limpio['TLI']
boxscore_limpio['2FGM'] = boxscore_limpio['2PC']
boxscore_limpio['2FGA'] = boxscore_limpio['2PI']
boxscore_limpio['3FGM'] = boxscore_limpio['3PC']
boxscore_limpio['3FGA'] = boxscore_limpio['3PI']

# ✅ Verificación de columnas clave
print("\n🔍 Columnas disponibles después de crear alias:")
print([col for col in boxscore_limpio.columns if col in ['FGM', 'FGA', 'FTM', 'FTA', '2FGM', '2FGA', '3FGM', '3FGA']])



✅ PASO 11 - Renombrado de columnas completado.

🔍 Columnas disponibles después de crear alias:
['FGM', 'FGA', 'FTM', 'FTA', '2FGM', '2FGA', '3FGM', '3FGA']


In [28]:
# encabezadp de boxscore_limpio

boxscore_limpio.head()

,Jugadores,MIN,PTS,OREB,DREB,REB,AST,FP,PER,ROB,...,TLC,TLI,FGM,FGA,FTM,FTA,2FGM,2FGA,3FGM,3FGA
0,Melody Osuna,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Florencia Fernandez *,28.57,19.0,5.0,7.0,12.0,1.0,3.0,2.0,1.0,...,7.0,9.0,6.0,12.0,7.0,9.0,6.0,12.0,0.0,0.0
2,Stephanie Smidt,3.00,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,Julia Fernandez *,16.53,4.0,1.0,3.0,4.0,0.0,4.0,5.0,2.0,...,0.0,0.0,2.0,6.0,0.0,0.0,2.0,6.0,0.0,0.0
4,Candela Foresto *,26.53,8.0,2.0,1.0,3.0,3.0,2.0,1.0,1.0,...,0.0,0.0,3.0,5.0,0.0,0.0,1.0,3.0,2.0,2.0


In [29]:
# 🛠️ PASO 12: Cálculo de Estadísticas Avanzadas
def stats_advanced(df):
    """ Calcula métricas avanzadas de rendimiento en baloncesto. """

    # Repetición de alias (por seguridad en uso independiente)
    df['FGM'] = df['TCC']
    df['FGA'] = df['TCI']
    df['FTM'] = df['TLC']
    df['FTA'] = df['TLI']
    df['2FGM'] = df['2PC']
    df['2FGA'] = df['2PI']
    df['3FGM'] = df['3PC']
    df['3FGA'] = df['3PI']

    # Columnas auxiliares si faltan
    if 'TO' not in df.columns:
        df['TO'] = 0
    if 'OFF REB' not in df.columns:
        df['OFF REB'] = df['OREB']
    if 'DEF REB' not in df.columns:
        df['DEF REB'] = df['DREB']

    # Totales por partido/equipo
    TIME_TOT = df.groupby(['Match_ID', 'Team'])['MIN'].transform('sum')
    FGM_TOT = df.groupby(['Match_ID', 'Team'])['FGM'].transform('sum')
    FGA_TOT = df.groupby(['Match_ID', 'Team'])['FGA'].transform('sum')
    FTA_TOT = df.groupby(['Match_ID', 'Team'])['FTA'].transform('sum')
    TO_TOT = df.groupby(['Match_ID', 'Team'])['TO'].transform('sum')
    RO_TOT = df.groupby(['Match_ID', 'Team'])['OFF REB'].transform('sum')
    RD_TOT = df.groupby(['Match_ID', 'Team'])['DEF REB'].transform('sum')

    # Totales del oponente
    RD_TOT_OPP = df.groupby(['Match_ID', 'Opp'])['DEF REB'].sum().to_dict()
    RO_TOT_OPP = df.groupby(['Match_ID', 'Opp'])['OFF REB'].sum().to_dict()

    df['Opp DEF REB Total'] = df.set_index(['Match_ID', 'Team']).index.map(RD_TOT_OPP)
    df['Opp OFF REB Total'] = df.set_index(['Match_ID', 'Team']).index.map(RO_TOT_OPP)

    # Métricas avanzadas
    df['RO%'] = (df['OFF REB'] * (TIME_TOT / 5) / (df['MIN'] * (RO_TOT + df['Opp DEF REB Total']))).round(2)
    df['RD%'] = (df['DEF REB'] * (TIME_TOT / 5) / (df['MIN'] * (RD_TOT + df['Opp OFF REB Total']))).round(2)
    df['AST %'] = (df['AST'] / (((df['MIN'] / (TIME_TOT / 5)) * FGM_TOT) - df['FGM'])).round(2)
    df['USG %'] = ((df['FGA'] + 0.44 * df['FTA'] + df['TO']) * (TIME_TOT / 5) / (df['MIN'] * (FGA_TOT + 0.44 * FTA_TOT + TO_TOT))).round(2)
    df['PLAYS'] = (df['FGA'] + 0.44 * df['FTA'] + df['TO'])
    df['PPP'] = (df['PTS'] / df['PLAYS']).round(3)

    df['FG %'] = (df['FGM'] / df['FGA']).round(2)
    df['FT %'] = (df['FTM'] / df['FTA']).round(2)
    df['2FG %'] = (df['2FGM'] / df['2FGA']).round(2)
    df['3FG %'] = (df['3FGM'] / df['3FGA']).round(2)
    df['TS %'] = (df['PTS'] / (2 * (df['FGA'] + 0.44 * df['FTA']))).round(2)
    df['EFG %'] = ((df['FGM'] + 0.5 * df['3FGM']) / df['FGA']).round(2)
    df['TO %'] = (df['TO'] / df['PLAYS']).round(2)
    df['PPT1'] = (df['FTM'] / df['FTA']).round(2)
    df['PPT2'] = (df['2FGM'] * 2 / df['2FGA']).round(2)
    df['PPT3'] = (df['3FGM'] * 3 / df['3FGA']).round(2)
    df['F1 T1 Plays %'] = (df['FTA'] * 0.44 / df['PLAYS']).round(2)
    df['F2 T2 PLAYS %'] = (df['2FGA'] / df['PLAYS']).round(2)
    df['F3 T3 PLAYS %'] = (df['3FGA'] / df['PLAYS']).round(2)
    df['F4 PP PLAYS %'] = (df['TO'] / df['PLAYS']).round(2)
    df['RTL %'] = (df['FTM'] / df['FGA']).round(2)

    # Limpieza final
    df.drop(columns=['Opp DEF REB Total', 'Opp OFF REB Total'], inplace=True)
    df = df[df['Jugadores'] != 'Sin asignar'].fillna(0)

    print("\n✅ PASO 12 - Cálculo de estadísticas avanzadas completado.")
    return df


In [30]:
# 📂 Guardar boxscore_advanced en CSV
print("📤 Exportando estadísticas avanzadas...")

import datetime, os

boxscore_advanced = stats_advanced(boxscore_limpio)
fecha_actual = datetime.datetime.now().strftime("%Y-%m-%d")
nombre_archivo = f"stats_advance_{fecha_actual}.csv"
ruta_completa = os.path.join("data", nombre_archivo)

boxscore_advanced.to_csv(ruta_completa, index=False, encoding='utf-8-sig')
print(f"\n✅ Archivo guardado correctamente en:\n{ruta_completa}")


📤 Exportando estadísticas avanzadas...

✅ PASO 12 - Cálculo de estadísticas avanzadas completado.

✅ Archivo guardado correctamente en:
data\stats_advance_2025-04-21.csv


In [31]:
# 🛠️ PASO 12: Cálculo de estadísticas avanzadas (versión refactorizada)
def stats_advanced(df):
    """ Calcula métricas avanzadas con nombres estandarizados desde boxscore_limpio.csv """

    # Alias internos para compatibilidad con nomenclatura tradicional
    df['FGM'] = df['TCC']
    df['FGA'] = df['TCI']
    df['FTM'] = df['TLC']
    df['FTA'] = df['TLI']
    df['2FGM'] = df['2PC']
    df['2FGA'] = df['2PI']
    df['3FGM'] = df['3PC']
    df['3FGA'] = df['3PI']
    df['OFF REB'] = df['OREB']
    df['DEF REB'] = df['DREB']
    df['IdMatch'] = df['Match_ID']  # Crear columna para mantener compatibilidad si usas 'IdMatch'

    # Calcular totales por partido y equipo
    TIME_TOT = df.groupby(['IdMatch', 'Team'])['MIN'].transform('sum')
    FGM_TOT = df.groupby(['IdMatch', 'Team'])['FGM'].transform('sum')
    FGA_TOT = df.groupby(['IdMatch', 'Team'])['FGA'].transform('sum')
    FTA_TOT = df.groupby(['IdMatch', 'Team'])['FTA'].transform('sum')
    TO_TOT = df.groupby(['IdMatch', 'Team'])['TO'].transform('sum') if 'TO' in df.columns else 0
    RO_TOT = df.groupby(['IdMatch', 'Team'])['OFF REB'].transform('sum')
    RD_TOT = df.groupby(['IdMatch', 'Team'])['DEF REB'].transform('sum')

    # Rebotes del oponente
    RD_TOT_OPP = df.groupby(['IdMatch', 'Opp'])['DEF REB'].sum().to_dict()
    RO_TOT_OPP = df.groupby(['IdMatch', 'Opp'])['OFF REB'].sum().to_dict()
    df['Opp DEF REB Total'] = df.set_index(['IdMatch', 'Team']).index.map(RD_TOT_OPP)
    df['Opp OFF REB Total'] = df.set_index(['IdMatch', 'Team']).index.map(RO_TOT_OPP)

    # Cálculo de métricas avanzadas
    df['RO%'] = (df['OFF REB'] * (TIME_TOT / 5) / (df['MIN'] * (RO_TOT + df['Opp DEF REB Total']))).round(2)
    df['RD%'] = (df['DEF REB'] * (TIME_TOT / 5) / (df['MIN'] * (RD_TOT + df['Opp OFF REB Total']))).round(2)
    df['AST %'] = (df['AST'] / (((df['MIN'] / (TIME_TOT / 5)) * FGM_TOT) - df['FGM'])).round(2)
    df['USG %'] = ((df['FGA'] + 0.44 * df['FTA'] + df['TO']) * (TIME_TOT / 5) / (df['MIN'] * (FGA_TOT + 0.44 * FTA_TOT + TO_TOT))).round(2)
    df['PLAYS'] = (df['FGA'] + 0.44 * df['FTA'] + df['TO'])
    df['PPP'] = (df['PTS'] / df['PLAYS']).round(3)

    # Porcentajes de tiro
    df['FG %'] = (df['FGM'] / df['FGA']).round(2)
    df['FT %'] = (df['FTM'] / df['FTA']).round(2)
    df['2FG %'] = (df['2FGM'] / df['2FGA']).round(2)
    df['3FG %'] = (df['3FGM'] / df['3FGA']).round(2)
    df['TS %'] = (df['PTS'] / (2 * (df['FGA'] + 0.44 * df['FTA']))).round(2)
    df['EFG %'] = ((df['FGM'] + 0.5 * df['3FGM']) / df['FGA']).round(2)
    df['TO %'] = (df['TO'] / df['PLAYS']).round(2)
    df['PPT1'] = (df['FTM'] / df['FTA']).round(2)
    df['PPT2'] = (df['2FGM'] * 2 / df['2FGA']).round(2)
    df['PPT3'] = (df['3FGM'] * 3 / df['3FGA']).round(2)
    df['F1 T1 Plays %'] = (df['FTA'] * 0.44 / df['PLAYS']).round(2)
    df['F2 T2 PLAYS %'] = (df['2FGA'] / df['PLAYS']).round(2)
    df['F3 T3 PLAYS %'] = (df['3FGA'] / df['PLAYS']).round(2)
    df['F4 PP PLAYS %'] = (df['TO'] / df['PLAYS']).round(2)
    df['RTL %'] = (df['FTM'] / df['FGA']).round(2)

    # Limpieza final
    df = df.drop(columns=['Opp DEF REB Total', 'Opp OFF REB Total'])
    df = df[df['Jugadores'] != 'Sin asignar']
    df = df.fillna(0)

    print("\n✅ PASO 12 - Estadísticas avanzadas calculadas correctamente.")
    return df


In [46]:
# 🛠️ PASO 12: Calcular y guardar boxscore_final con estadísticas avanzadas

# Asegúrate de tener definidos boxscore_limpio y la función stats_advanced()
print("🔢 Calculando estadísticas avanzadas...")

# Generar DataFrame final con métricas
boxscore_final = stats_advanced(boxscore_limpio)

# Mostrar las primeras 15 filas como verificación
print("\n✅ Primeras 15 filas de boxscore_final:")
display(boxscore_final.head(15))

# 📂 Guardar en carpeta /data/ con fecha


nombre_archivo = f"boxscore_final.csv"
ruta_csv = os.path.join("data", nombre_archivo)

boxscore_final.to_csv(ruta_csv, index=False, encoding='utf-8-sig')
print(f"\n📁 Archivo guardado correctamente en: {ruta_csv}")


🔢 Calculando estadísticas avanzadas...

✅ PASO 12 - Estadísticas avanzadas calculadas correctamente.

✅ Primeras 15 filas de boxscore_final:


,Jugadores,MIN,PTS,OREB,DREB,REB,AST,FP,PER,ROB,...,TO %,PPT1,PPT2,PPT3,F1 T1 Plays %,F2 T2 PLAYS %,F3 T3 PLAYS %,F4 PP PLAYS %,RTL %,IdMatch
0,Melody Osuna,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,118314-FCO-SSI
1,Florencia Fernandez *,28.57,19.0,5.0,7.0,12.0,1.0,3.0,2.0,1.0,...,0.0,0.78,1.00,0.00,0.25,0.75,0.00,0.0,0.58,118314-FCO-SSI
2,Stephanie Smidt,3.00,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,...,0.0,0.00,0.00,0.00,0.00,0.00,1.00,0.0,0.00,118314-FCO-SSI
3,Julia Fernandez *,16.53,4.0,1.0,3.0,4.0,0.0,4.0,5.0,2.0,...,0.0,0.00,0.67,0.00,0.00,1.00,0.00,0.0,0.00,118314-FCO-SSI
4,Candela Foresto *,26.53,8.0,2.0,1.0,3.0,3.0,2.0,1.0,1.0,...,0.0,0.00,0.67,3.00,0.00,0.60,0.40,0.0,0.00,118314-FCO-SSI
5,Mora Sauro,1.43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.00,1.00,0.00,0.0,0.00,118314-FCO-SSI
6,Abril Romagnoli *,14.55,2.0,1.0,0.0,1.0,1.0,4.0,2.0,1.0,...,0.0,0.00,1.00,0.00,0.00,0.33,0.67,0.0,0.00,118314-FCO-SSI
7,Lucila Sampietro,19.15,0.0,1.0,1.0,2.0,0.0,1.0,1.0,1.0,...,0.0,0.00,0.00,0.00,0.00,0.45,0.55,0.0,0.00,118314-FCO-SSI
8,Maribel Barzola,20.25,3.0,0.0,4.0,4.0,1.0,1.0,2.0,1.0,...,0.0,0.00,0.00,0.75,0.00,0.20,0.80,0.0,0.00,118314-FCO-SSI
9,Julieta Perez,14.65,7.0,3.0,7.0,10.0,0.0,2.0,2.0,1.0,...,0.0,0.50,0.80,0.00,0.35,0.65,0.00,0.0,0.60,118314-FCO-SSI



📁 Archivo guardado correctamente en: data\boxscore_final.csv


## GRAFICAS AVANZADAS PARA JUGADORAS Y EQUIPOS

In [45]:
tripleras = boxscore_final[boxscore_final['3FGA'] > 0]
fig = px.bar(
    tripleras.sort_values('3FG %', ascending=False).head(10),
    x='Jugadores', y='3FG %', color='Team',
    title='Top 10 Jugadoras con Mayor Efectividad en Triples (3FG%)'
)
